In [1]:
import jaydebeapi
import numpy
import pandas

<h3>環境設置</h3>

In [2]:
stock_ID="NQF"
data_table="stockInfo"

In [3]:
field_names=[
    'StockID', 'StockDate', 
    'StockOpen', 'StockHigh', 'StockLow', 'StockClose', 
    'StockAdjClose', 'StockVol'
    ]

<h3>開啟資料庫</h3>

In [2]:
dbConnection =jaydebeapi.connect(
    "org.h2.Driver",
    "jdbc:h2:/Users/justin/Project/h2",
    ["sa",""],
    "/Users/justin/Project/h2/bin/h2-2.1.214.jar"
)
dbCursor = dbConnection.cursor()
#作業空間

<h3>資料統計-按年月小計</h3>

<h3>最大,最小值,平均數</h3>

In [ ]:
#用SQL做
_sql=[
    "SELECT LEFT(StockDate,7) AS YM,", #字串函數(LEFT),前七位
    "MIN(StockOpen) AS MinstockOpen,AVG(StockOpen) AS AvgstockOpen,MAX(StockOpen) AS MaxstockOpen",
    "FROM STOCKINFO",
    "WHERE StockID= '%s'" % stock_ID,
    "GROUP BY LEFT(StockDate,7)" #小計
]

dbCursor.execute(' '.join(_sql)) #執行sql語句
resultSet = dbCursor.fetchall() #tuple資料  #获取查询返回结果

rows={
    'YM':[],
    'MIN_OPEN':[],
    'AVG_OPEN':[],
    'MAX_OPEN':[]    
}
#資料結構(陣列),初始化

for row in resultSet:
    rows['YM'].append(row[0]),
    rows['MIN_OPEN'].append(row[1])
    rows['AVG_OPEN'].append(row[2])
    rows['MAX_OPEN'].append(row[3])

results=pandas.DataFrame(data=rows).set_index(['YM'] )#把row指派到data裡面
results.head()

In [ ]:
#用python做
_sql=[
    "SELECT STOCKDATE ,LEFT(StockDate,7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE StockID= '%s'" % stock_ID
]
#資料量較大,運算負荷較少
dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows={
    'STOCKDATE':[],
    'YM':[],
    'STOCKOPEN':[]
}
for row in resultSet:
    rows['STOCKDATE'].append(row[0])
    rows['YM'].append(row[1])
    rows['STOCKOPEN'].append(row[2])
    
results=pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min','mean','max']) #agg計算統計量
results.columns=[['MIN_OPEN','AVG_OPEN','MAX_OPEN']]                                                                         
results.head()

In [ ]:
#針對年份(2019,2020)
_sql=[
    "SELECT LEFT(StockDate,7) AS YM,",
    "MIN(StockOpen) AS MinstockOpen,AVG(StockOpen) AS AvgstockOpen,MAX(StockOpen) AS MaxstockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE,4)='2019' OR LEFT(STOCKDATE,4)='2020' AND StockID= '%s'" % stock_ID, #更改條件
    "GROUP BY LEFT(StockDate,7)"
]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows={
    'YM':[],
    'MIN_OPEN':[],
    'AVG_OPEN':[],
    'MAX_OPEN':[]    
}

for row in resultSet:
    rows['YM'].append(row[0]),
    rows['MIN_OPEN'].append(row[1])
    rows['AVG_OPEN'].append(row[2])
    rows['MAX_OPEN'].append(row[3])

results=pandas.DataFrame(data=rows).set_index(['YM'])
results

In [ ]:
#宣告年份變數
_year=2018
_sql=[
    "SELECT STOCKDATE ,LEFT(StockDate,7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE,4)='%s' AND StockID= '%s'" % (_year,stock_ID), #更改指派條件
    ]

dbCursor.execute(' '.join(_sql))
resultSet = dbCursor.fetchall()

rows={
    'STOCKDATE':[],
    'YM':[],
    'STOCKOPEN':[]
}

for row in resultSet:
    rows['STOCKDATE'].append(row[0])
    rows['YM'].append(row[1])
    rows['STOCKOPEN'].append(row[2])

results=pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min','mean','max'])
results.columns=[['MIN_OPEN','AVG_OPEN','MAX_OPEN']]                                                                                   
results.head()

<h3>設計函式</h3>

In [ ]:
def Calc_Open(year,stock_id): #記得縮排,注意大小寫
    _sql=[
    "SELECT STOCKDATE ,LEFT(StockDate,7) AS YM, StockOpen",
    "FROM STOCKINFO",
    "WHERE LEFT(STOCKDATE,4)='%s' AND StockID= '%s'" % (year,stock_id), #更改指派條件
    ]

    dbCursor.execute(' '.join(_sql))
    resultSet = dbCursor.fetchall()

    rows={
        'STOCKDATE':[],
        'YM':[],
        'STOCKOPEN':[]
    }

    for row in resultSet:
        rows['STOCKDATE'].append(row[0])
        rows['YM'].append(row[1])
        rows['STOCKOPEN'].append(row[2])

    results=pandas.DataFrame(data=rows).set_index(['STOCKDATE']).groupby(by=['YM']).agg(['min','mean','max'])
    results.columns=[['MIN_OPEN','AVG_OPEN','MAX_OPEN']]           
    
    return results #return結果

In [ ]:
df_2019=Calc_Open(year=2019,stock_id='NQF')
df_2019.tail()

In [ ]:
df_all=[]
for _year in [2017,2018,2019]:
    _df=Calc_Open(year=_year,stock_id='IBM')
    df_all.append(_df)
df_all

In [3]:
dbCursor.close()
dbConnection.close()

NameError: name 'dbCursor' is not defined